In [1]:
from task import * 
# from utils import X,y,cat_cols,fillna,normalize,skew,encode

In [2]:
import gc


In [3]:
def data():
    return ( pd.read_csv('datasets/train.csv') )

def cross_valid_sets():
    df0 = data()
    df0,df2 = train_test_split(df0,test_size=.5)
    df0,df1 = train_test_split(df0,test_size=.5)
    df2,df3 = train_test_split(df2,test_size=.5)
    return([df0,df1,df2,df3])


### Класс для кроссвалидации коррекций данных и гиперпараметров
Из-за ограничений по объему памяти и времени выполнения  проверяем толькр наиболее интересные варианты 

In [4]:
class CrossValidator(Task):

    def __init__(self, dfs,params = {}):
        super().__init__(dfs,params)
        self.datasets = []
        for i in [0,1,2,3]:
            dx = shuffle(pd.concat( dfs[:i]+dfs[(i+1):] ,axis=0 )).copy()
            self.datasets.append(ScoreSet( dx, dfs[i].copy(),params=self.params) ) 


    def process_fold(self,f):
        self.log +=(f"{f.log} \n")  
        f.set_encoder( self.new_encoder() ).set_model(self.new_model() )
        f.prepare().fit().predict()
        self.log +=(f"{f.log} \n")  
        
    def process(self):
        for f in self.datasets: 
            self.process_fold(f) 
        self.score = self.mean_score()   
        print(f"MAPE mean: {self.score}")    
        return(self)
    
    def mean_score(self):
        return sum( [ds.scores() for ds in self.datasets ])/len(self.datasets)
    

In [5]:
params = { 
            'rounding': { 'odometer':3000,'condition':1},
            'model': {  'criterion':'squared_error','n_estimators':20, 'max_depth':4 }    
        }
cv = CrossValidator( cross_valid_sets() ,params=params)
cv.process()
del cv
gc.collect()

MAPE mean: 0.3022824299217073


1326